# Forecasting Carbon Intensity using SARIMA #

## Importing modules ##

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

## Processing data for monthly carbon intensity ##

In [3]:
df = pd.read_csv('../data/df_fuel_ckan.csv')
df['DATETIME'] = pd.to_datetime(df['DATETIME'])
df_filtered = df[df['DATETIME'].dt.year < 2025].copy()
df_filtered.set_index('DATETIME', inplace=True)
data_train = df_filtered['CARBON_INTENSITY'].resample('M').mean().dropna()
data_train.head()

C:\Users\tianw\AppData\Local\Temp\ipykernel_14380\4079819580.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data_train = df_filtered['CARBON_INTENSITY'].resample('M').mean().dropna()


DATETIME
2009-01-31    578.405242
2009-02-28    534.950893
2009-03-31    464.442876
2009-04-30    408.872917
2009-05-31    409.803091
Freq: ME, Name: CARBON_INTENSITY, dtype: float64

## Fitting a SARIMA model to the data ##

In [4]:
model = SARIMAX(data_train, order=(1, 1, 1), 
seasonal_order=(1, 1, 1, 12), 
enforce_stationarity=False,
enforce_invertibility=False)
model_fit = model.fit(disp=False)
print(model_fit.summary())

                                     SARIMAX Results                                      
Dep. Variable:                   CARBON_INTENSITY   No. Observations:                  192
Model:             SARIMAX(1, 1, 1)x(1, 1, 1, 12)   Log Likelihood                -785.731
Date:                            Sat, 13 Dec 2025   AIC                           1581.463
Time:                                    22:15:21   BIC                           1596.992
Sample:                                01-31-2009   HQIC                          1587.767
                                     - 12-31-2024                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.3680      0.129      2.849      0.004       0.115       0.621
ma.L1         -0.7508      0.096   